In [ ]:
import os
import pathlib
import yaml

from tensorboard.backend.event_processing import event_accumulator
import pandas as pd

import matplotlib.pyplot as plt

In [ ]:
cwd = pathlib.Path(os.getcwd())
with open(cwd.parent / 'config_tyndall.yaml') as stream:
    config = yaml.safe_load(stream)

config['Directories']['work_dir'] = pathlib.Path(config['Directories']['work_dir'])
for key, value in config['Directories'].items():
    if key == 'work_dir' or key == 'color_to_phase_dicts':
        continue
    else:
        config['Directories'][key] = config['Directories']['work_dir'] / value

In [ ]:
def parse_tensorboard(path):
    """returns a dictionary of pandas dataframes for all scalars"""
    ea = event_accumulator.EventAccumulator(path, size_guidance={event_accumulator.SCALARS: 0})
    ea.Reload()
    scalars = ea.Tags()['scalars']
    train_metrics = {}
    val_metrics = {}
    grad_data = {}
    for tag in sorted(scalars):
        if 'train' in tag and 'epoch' in tag:
            values = []
            for scalar_event in ea.Scalars(tag):
                values.append(scalar_event.value)
            train_metrics[tag] = values
        elif 'val' in tag:
            values = []
            for scalar_event in ea.Scalars(tag):
                values.append(scalar_event.value)
            val_metrics[tag] = values
        elif 'conv' in tag:
            gradients = []
            for scalar_event in ea.Scalars(tag):
                gradients.append(scalar_event.value)
            grad_data[tag] = gradients

    train_metrics = pd.DataFrame(data = train_metrics)
    val_metrics = pd.DataFrame(data = val_metrics)
    gradients = pd.DataFrame(data = grad_data)
    return train_metrics, val_metrics, gradients 
    

In [ ]:
log_path = list((config['Directories']['logs_dir'] / config['Model_Details']['name']).rglob('version_0/events*'))
train_logs, val_logs, gradients = parse_tensorboard(str(log_path[0]))

In [ ]:
fig = plt.figure(figsize = (10, 5), tight_layout = True)

ax1 = fig.add_subplot(2, 3, 1)
ax1.set_title('Losses')
train_logs[['train_loss_epoch']].plot(ax = ax1)
val_logs[['val_loss']].plot(ax = ax1)           

ax2 = fig.add_subplot(2, 3, 2)
ax2.set_title('ODS F1 score')
train_logs[['train_ODSF1_epoch']].plot(ax = ax2)
val_logs[['val_ODSF1']].plot(ax = ax2)   

ax3 = fig.add_subplot(2, 3, 3)
ax3.set_title('OIS F1 scores')
train_logs[['train_OISF1_epoch']].plot(ax = ax3)
val_logs[['val_OISF1']].plot(ax = ax3)   

ax4 = fig.add_subplot(2, 3, 4)
ax4.set_title('AP')
train_logs[['train_AveragePrecision_epoch']].plot(ax = ax4)
val_logs[['val_AveragePrecision']].plot(ax = ax4)   

ax5 = fig.add_subplot(2, 3, 5)
ax5.set_title('IOU')
train_logs[['train_IOU_epoch']].plot(ax = ax5)
val_logs[['val_IOU']].plot(ax = ax5)   